# Sigma Protocols

**Module 09** | 09-commitments-sigma-protocols

*3-move structure, soundness, ZK*

## Objectives

By the end of this notebook you will be able to:

1. Understand the core ideas behind **sigma protocols**.
2. Explore these concepts interactively using SageMath.
3. Build intuition through hands-on computation and visualization.

## Prerequisites

- Completion of [Pedersen Commitments](09b-pedersen-commitments.ipynb).
- Concepts and notation introduced in the previous notebook.

## The 3-Move Structure

In [ ]:
# Prover -> Verifier: commitment (a)
# Verifier -> Prover: challenge (e)
# Prover -> Verifier: response (z)
# TODO: diagram the 3-move protocol

## Soundness

In [ ]:
# A cheating prover cannot answer two different challenges
# Special soundness: from two accepting transcripts, extract witness
# TODO: illustrate soundness extraction

## Zero-Knowledge Property

In [ ]:
# The verifier learns nothing beyond the statement's truth
# Simulator: can produce valid-looking transcripts without the witness
# TODO: demonstrate the simulation argument

## Exercises

Try these on your own before moving on:

1. **Exercise 1:** *(TODO: add exercise)*
2. **Exercise 2:** *(TODO: add exercise)*
3. **Exercise 3:** *(TODO: add exercise)*

## Summary

In this notebook we explored **sigma protocols**.  Key takeaways:

- *(TODO: summarize key point 1)*
- *(TODO: summarize key point 2)*
- *(TODO: summarize key point 3)*

**Next:** [The Schnorr Protocol](09d-schnorr-protocol.ipynb)